# Librerias


In [ ]:
!pip install pythonds

# Importaciones

In [ ]:
from pythonds.basic.stack import Stack

import pydot
from PIL import Image
from IPython.display import display

# Parseo de la expresion regular

In [ ]:
def toPostfix(regularExpression):
    operatorPrecedence = {'(': 0, 'U': 1, '.': 2, '+': 3, '*': 4}
    alphabet = 'abcd'
    operators = 'U.+*'

    output = ''
    operatorStack = Stack()

    for token in regularExpression:
        if token in operators:
            while(not operatorStack.isEmpty() and \
              operatorStack.peek() != '(' and \
              operatorPrecedence[operatorStack.peek()] >= operatorPrecedence[token]):
                output += operatorStack.pop()
            operatorStack.push(token)
        elif token == '(':
            operatorStack.push(token)
        elif token == ')':
            while operatorStack.peek() != '(':
                output += operatorStack.pop()
            operatorStack.pop()
        else:
            output += token

    while not operatorStack.isEmpty():
        output += operatorStack.pop()

    return output

def insertDotOperator(regularExpression):
    operators = 'U.+*'
    output = ''

    for index, token in enumerate(regularExpression):
        output += token

        if token == '(' or token == 'U' or token == '.':
            continue
        if index < len(regularExpression) - 1:
            lookahead = regularExpression[index + 1]
            if lookahead in operators or lookahead == ')':
                continue
            output += '.'

    return output

# Procesamiento de la expresion regular parseada


In [ ]:
class NFA:
    def __init__(self, states, alphabet, transitions, initialState, finalState):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.initialState = initialState
        self.finalState = finalState

def getMaxMinStates(leftNFA, rightNFA = None):
    if rightNFA is not None:
        states = leftNFA.states.union(rightNFA.states)
    else:
        states = leftNFA.states

    return min(states), max(states)

def makeConcatenation(leftNFA, rightNFA):
    states = leftNFA.states.union(rightNFA.states)
    alphabet = leftNFA.alphabet.union(rightNFA.alphabet)

    transitions = {}
    transitions.update(leftNFA.transitions)
    transitions.update(rightNFA.transitions)
    transitions.update({(leftNFA.finalState, ''): rightNFA.initialState})

    initialState = leftNFA.initialState
    finalState = rightNFA.finalState

    return NFA(states, alphabet, transitions, initialState, finalState)

def makeUnion(leftNFA, rightNFA):
    minState, maxState = getMaxMinStates(leftNFA, rightNFA)
    initialState = minState
    finalState = maxState + 2

    states = {initialState, finalState}
    for state in leftNFA.states:
        states.add(state + 1)

    for state in rightNFA.states:
        states.add(state + 1)

    alphabet = leftNFA.alphabet.union(rightNFA.alphabet)

    transitions = {}
    transitions.update(leftNFA.transitions)
    transitions.update(rightNFA.transitions)

    newTransitions = {}
    for key, value in transitions.items():
        lst = list(key)
        lst[0] = lst[0] + 1
        key = tuple(lst)
        newTransitions[key] = value + 1
    transitions = newTransitions

    transitions.update({(leftNFA.finalState + 1, ''): finalState})
    transitions.update({(rightNFA.finalState + 1, ''): finalState})

    transitions.update({(initialState, '', initialState + 1): leftNFA.initialState + 1})
    transitions.update({(initialState, '', initialState + 2): rightNFA.initialState + 1})

    return NFA(states, alphabet, transitions, initialState, finalState)

def makeStar(nfa):
    minState, maxState = getMaxMinStates(nfa)
    initialState = minState
    finalState = maxState + 2

    states = {initialState, finalState}
    for state in nfa.states:
        states.add(state + 1)

    alphabet = nfa.alphabet

    transitions = {}
    transitions.update(nfa.transitions)

    newTransitions = {}
    for key, value in transitions.items():
        lst = list(key)
        lst[0] = lst[0] + 1
        key = tuple(lst)
        newTransitions[key] = value + 1
    transitions = newTransitions

    transitions.update({(initialState, '', initialState + 1): nfa.initialState + 1})
    transitions.update({(initialState, '', initialState + 2): finalState})

    transitions.update({(nfa.finalState + 1, '', initialState + 1): nfa.initialState + 1})
    transitions.update({(nfa.finalState + 1, '', initialState + 2): finalState})

    return NFA(states, alphabet, transitions, initialState, finalState)

def makePlus(nfa):
    minState, maxState = getMaxMinStates(nfa)
    initialState = minState
    finalState = maxState + 2

    states = {initialState, finalState}
    for state in nfa.states:
        states.add(state + 1)

    alphabet = nfa.alphabet

    transitions = {}
    transitions.update(nfa.transitions)

    newTransitions = {}
    for key, value in transitions.items():
        lst = list(key)
        lst[0] = lst[0] + 1
        key = tuple(lst)
        newTransitions[key] = value + 1
    transitions = newTransitions

    transitions.update({(initialState, '', initialState + 1): nfa.initialState + 1})

    transitions.update({(nfa.finalState + 1, '', initialState + 1): nfa.initialState + 1})
    transitions.update({(nfa.finalState + 1, '', initialState + 2): finalState})

    return NFA(states, alphabet, transitions, initialState, finalState)

def makeBasicNFA(maxState, token):
    initialState = maxState + 1
    finalState = maxState + 2

    states = {initialState, finalState}
    alphabet = {token}
    transitions = {(initialState, token): finalState}

    return NFA(states, alphabet, transitions, initialState, finalState)

def makeNFAFromRE(regularExpression):
    stack = Stack()

    for token in regularExpression:
        if token == '.':
            rightNFA = stack.pop()
            leftNFA = stack.pop()
            stack.push(makeConcatenation(leftNFA, rightNFA))
        elif token == 'U':
            rightNFA = stack.pop()
            leftNFA = stack.pop()
            stack.push(makeUnion(leftNFA, rightNFA))
        elif token == '*':
            nfa = stack.pop()
            stack.push(makeStar(nfa))
        elif token == '+':
            nfa = stack.pop()
            stack.push(makePlus(nfa))
        else:
            if not stack.isEmpty():
                _, maxState = getMaxMinStates(stack.peek())
            else:
                maxState = 0
            stack.push(makeBasicNFA(maxState, token))

    return stack.pop()

# Graficacion

In [ ]:
def drawNFA(nfa):
    graph = pydot.Dot(graph_type='digraph')
    for state in nfa.states:
        if state == nfa.finalState:
            node = pydot.Node(str(state), shape='doublecircle', label=f'q{state}')
        elif state == nfa.initialState:
            node = pydot.Node(str(state), shape='circle', style='bold', label=f'q{state}')
        else:
            node = pydot.Node(str(state), shape='circle', label=f'q{state}')
        graph.add_node(node)

    for transicion, destino in nfa.transitions.items():
        if transicion[1] == '':
            edge = pydot.Edge(str(transicion[0]), str(destino), label='ε')
        else:
            edge = pydot.Edge(str(transicion[0]), str(destino), label=transicion[1])
        graph.add_edge(edge)

    graph.write_png('./afn.png')
    im = Image.open('./afn.png')
    display(im)

# Pruebas


In [ ]:
# Se parsea la expresion regular y se convierte a su forma Sufija
regularExpression = input("Enter the regular expression: ")
regularExpression = insertDotOperator(regularExpression)
regularExpression = toPostfix(regularExpression)

# Construye el AFND
nfa = makeNFAFromRE(regularExpression)

# Muestra la representación textual del AFND
print("States:", nfa.states)
print("Alphabet:", nfa.alphabet)

print("Transitions:")
for key, value in nfa.transitions.items():
    print(f'   ({key[0]}, {key[1]}) -> {value}')

print("Initial State:", nfa.initialState)
print("Final State:", nfa.finalState)

# Dibuja el AFND
drawNFA(nfa)